In [ ]:
# default_exp signUp

# Sign up

> sign up

In [ ]:
#hide
import src.setEnv

In [ ]:
#export
from beartype import beartype
from awsSchema.apigateway import Response, Event
from dataclasses import dataclass
from dataclasses_json import dataclass_json
from copy import deepcopy

In [ ]:
#export
from src import passwordHelper as PH
from src.passwordTable import UserPasswordTable

## User input class

In [ ]:
#export
@dataclass_json
@dataclass
class User:
  username:str
  password:str
    
  @property
  def passwordHash(self):
    return PH.hashPassword(self.password)
  
  def save(self):
    table = UserPasswordTable(
      username = self.username,
      passwordHash = self.passwordHash
    )
    table.save()
  

In [ ]:
# SetUpUsername(username = '123', password='123')
sun = User.from_dict( {'username': 'nic', 'password':'123'})
sun.passwordHash
sun.save()

nic - bf353dc2b0d01391d965b5e880cc48e229bb170afa09a36f4f0faa3ed0362ee6:53a67c26c6694af5a5e45f3675c3ed9c


## helper functions

In [ ]:
#export
class H:
  class ParseInputError(Exception): pass
  class SavingUserError(Exception): pass
  @classmethod
  @beartype
  def parseInput(cls,event:dict)->User:
    try:
      user = Event.parseDataClass(User,deepcopy(event))
      return user
    except Exception as e:
      raise cls.ParseInputError(e)
  
  @classmethod
  @beartype
  def save(cls, user:User)->bool:
    try:
      user.save()
      return True
    except Exception as e:
      raise cls.SavingUserError(e)
      
    
      

## main lambda handler

In [ ]:
#export
def signUp(event, *args):
  try:
    user = H.parseInput(event)
    H.save(user)
    return Response.returnSuccess()
  except H.SavingUserError as e:
    return Response.returnError(f'failed saving user {e}')
  except Exception as e:
    return Response.returnError(f'unknown error {e}')

In [ ]:
input_ = {'username': 'nic1', 'password':'123'}
event = Event.getInput(body = input_)
signUp(event)

nic1 - 20e3f405277f5c5a9c27a2298f6e4cd1aa0d5d508a5afa16258df052d48ecce7:08830fb8668a41799aee7113a6bf4625


{'body': '{}',
 'statusCode': 200,
 'headers': {'Access-Control-Allow-Headers': '*',
  'Access-Control-Allow-Origin': '*',
  'Access-Control-Allow-Methods': '*'}}